In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import torchvision

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [3]:
# from lutils.fin.autoencoder import Normalizer

In [4]:
# np.set_printoptions(precision=8, suppress=True)

In [5]:
# %matplotlib inline
%matplotlib qt

In [6]:
n_epochs = 40 #100   # number of epochs to run
batch_size = 8  # size of each batch
seq_size = 32

In [7]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, dropout=0.2):
#         super(FutureTrendEncoderLayer, self).__init__()
        
#         self.encoder_lstm = nn.Sequential(
#             nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         ) # batch_size hidden_layer_size * seq_size
        
#     def forward(self, x): # 8 32 1
#         lstm_out, (h_n, c_n) = self.encoder_lstm(x)
#         return lstm_out, h_n # 8 32 32, 2 8 32

# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self, input_size=33, hidden_layer_size=32, num_layers=2, dropout=0.2):
#         super(FutureTrendDecoderLayer, self).__init__()
        
#         self.lstm = nn.Sequential(
#             nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         )
        
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(1024, 1),
#         )
    
#     def forward(self, x, state): # 8 32 1, 2 8 32
#         batch_size = x.shape[0]
        
#         context = state[-1].repeat(x.shape[1], 1, 1).permute(1, 0, 2) # permute(1, 0, 2)  1, 8, 32  8, 32, 32
    
#         out = torch.cat((x, context), 2) # 32, 8, 48
#         out, (h_n, c_n) = self.lstm(out) # 8 32 32

#         return self.decoder_dense(out.reshape(batch_size, -1))

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 1
#         encoded, state = self.encoder_layer(x)
#         out = self.decoder_layer(x, state)
#         return out

In [8]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self, input_size=2, hidden_layer_size=32, num_layers=4, dropout=0.2):
#         super(FutureTrendEncoderLayer, self).__init__()
        
#         self.rnn = nn.Sequential(
#             nn.LayerNorm(input_size),
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True)
#         ) # batch_size hidden_layer_size * seq_size
        
#     def forward(self, x): # 8 32 4
#         rnn, h_n = self.rnn(x)
#         return rnn, h_n # 8 32 32, 4 8 32

# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self, input_size=34, hidden_layer_size=32, num_layers=4, dropout=0.2):
#         super(FutureTrendDecoderLayer, self).__init__()
        
#         self.rnn = nn.Sequential(
#             nn.LayerNorm(input_size),
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True)
#         )
        
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(1024, 1),
#         )
    
#     def forward(self, x, state): # 8 32 1, 2 8 32
#         batch_size = x.shape[0]
        
# #         context = state[-1].repeat(x.shape[1], 1, 1).permute(1, 0, 2) # permute(1, 0, 2)  1, 8, 32  8, 32, 32
# #         context = state[-1].reshape(batch_size, 32, 1)
# #         out = torch.cat((x, context), 2) # 32, 8, 48
        
#         out = torch.cat((x, state), 2)
#         out, h_n = self.rnn(out)
        
#         return self.decoder_dense(out.reshape(batch_size, -1))

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 4
#         encoded, h_n = self.encoder_layer(x) # torch.Size([8, 32, 32]) torch.Size([4, 8, 32])
#         out = self.decoder_layer(x, encoded)
#         return out, h_n

In [26]:
class FutureTrendEncoderLayer(nn.Module):
    def __init__(self, input_size=2, hidden_layer_size=32, num_layers=2, dropout=0.2):
        super(FutureTrendEncoderLayer, self).__init__()
        
        self.encoder_lstm = nn.Sequential(
            nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
        ) # batch_size hidden_layer_size * seq_size
        
    def forward(self, x): # 8 32 1
        rnn, h_n = self.encoder_lstm(x)
        return rnn, h_n # 8 32 32, 2 8 32

    
class FutureTrendAttention(nn.Module):
    def __init__(self):
        super(FutureTrendAttention, self).__init__()
    
    def forward(self, decoder_state, encoder_states):
        
        prob = nn.functional.softmax(encoder_states[-1] + decoder_state, dim=-1)
        context = torch.sum(prob.unsqueeze(-1) * encoder_states[-1].unsqueeze(-1), 1)
        
        return prob, context
        
    
class FutureTrendDecoderLayer(nn.Module):
    def __init__(self, input_size=2 , hidden_layer_size=32, num_layers=2, dropout=0.2):
        super(FutureTrendDecoderLayer, self).__init__()
        
#         self.lstm = nn.Sequential(
#             nn.GRUCell(input_size=input_size, hidden_size=hidden_layer_size)
#         )
        self.decoder_lstm = nn.Sequential(
            nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
        )
        
        self.attention = FutureTrendAttention()
        
        self.decoder_dense = nn.Sequential(
            nn.Linear(32, 1),
        )
    
    def forward(self, x, state): # 8 32 1, 2 8 32
        batch_size = x.shape[0]

        lstm, hn = self.decoder_lstm(x)
        prob, context = self.attention(hn[-1], state)
        
        return self.decoder_dense(prob), prob, context

class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        self.encoder_layer = FutureTrendEncoderLayer().to(0)
        self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
    def forward(self, x): # 8 32 1
        encoded, h_n = self.encoder_layer(x)
        out, prob, context = self.decoder_layer(x, h_n)
        return out, h_n, prob, context

In [27]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self):
#         super(FutureTrendEncoderLayer, self).__init__()
        
#         self.linear = nn.Linear(32, 256)
    
#         self.multihead_attn = nn.MultiheadAttention(8, 4, batch_first=True)
        
#     def forward(self, x): # 8 32 1
#         batch_size = x.size(0)
        
#         out = self.linear(x.reshape(batch_size, -1))
#         out = out.reshape(batch_size, 32, -1)
        
#         attn_output, attn_output_weights = self.multihead_attn(out, out, out) # 8 32 8, 8 32 32
# #         print(attn_output.shape)
# #         print(attn_output_weights.shape)
#         return attn_output, attn_output_weights

    
    
# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self):
#         super(FutureTrendDecoderLayer, self).__init__()
        
#         self.linear = nn.Linear(32, 256)
    
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(512, 1),
#         )
    
#     def forward(self, x, attn): # 8 32 1, 8 32 8
#         batch_size = x.shape[0]
#         linear = self.linear(x.reshape(batch_size, -1))
#         out = linear.reshape(batch_size, 32, -1)
        
#         out = torch.cat((out, attn), 0)
        
#         return self.decoder_dense(out.reshape(batch_size, -1)), linear

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 1
#         attn, attn_weights = self.encoder_layer(x)
#         out, linear = self.decoder_layer(x, attn)
#         return out, attn, attn_weights, linear
    

In [28]:
model = FutureTrendModule().to(0)
# model.load_state_dict(torch.load('D:/option/models/model_0619_autoencoder.pt'))

In [29]:
loss_fn = nn.MSELoss().to(0)  # mean square error
# loss_fn = nn.SmoothL1Loss().to(0)
# loss_fn = nn.CrossEntropyLoss().to(0)
# optimizer = optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.98), eps=1e-9)
# optimizer = optim.SGD(model.parameters(), lr = 0.000001, momentum=0.09)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) #

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
lambda1 = lambda epoch: np.power(0.1, epoch) if epoch <= 6 else 1e-9
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [30]:
DATA_PATH = '5S_10T'

In [31]:
# SAVE_PATH = os.path.join('D:/option/l8_autoencoder/', DATA_PATH)
# SAVE_PATH = os.path.join('D:/option/master/', DATA_PATH)

SAVE_PATH = os.path.join('D:/option/master/dims/diff', DATA_PATH)

In [32]:
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
        
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [33]:
# store = pd.HDFStore(os.path.join(SAVE_PATH, 'SHFE.rb.2021-04-06.5S.10T.41253.h5'), 'r')
# X_price = store['X_price']
# X_volume = store['X_volume']
# X_amount = store['X_amount']

# y = store['last_price']['last_price'].values

In [34]:
# d = np.stack((X_price.values.reshape([-1, 32, 1]), 
#               X_volume.values.reshape([-1, 32, 1]), 
#               X_amount.values.reshape([-1, 32, 1]), 
#               (X_amount / X_volume).values.reshape([-1, 32, 1])), axis=3).reshape(-1, 32, 4)

In [35]:
# scaler = StandardScaler()

In [36]:
# scaler.fit_transform(X_price)

In [37]:
# X_price

In [38]:
# d.shape

In [39]:
# d.shape

In [40]:
# scaler = StandardScaler()

In [41]:
writer = SummaryWriter('runs/log')

In [43]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

step = 0
# training loop
for epoch in range(n_epochs):

#     total_size = 0
    
#     for ix in range(0, len(dfs_x)-20):
    for j, f in enumerate(os.listdir(SAVE_PATH)):
        if not f.startswith('SHFE.rb'):
            continue
        
        store = pd.HDFStore(os.path.join(SAVE_PATH, f), 'r')
#         X = normalize(store['X'].values)
        X_price = store['X_price']
        X_volume = store['X_volume']
        X_amount = store['X_amount']
        
#         X_price = scaler.fit_transform(X_price)
#         X_volume = scaler.fit_transform(X_volume)
#         X_amount = scaler.fit_transform(X_amount)
        
        X = np.stack((X_price.values.reshape([-1, 32, 1]), 
                      X_volume.values.reshape([-1, 32, 1]), 
#                       X_amount.values.reshape([-1, 32, 1]), 
#                       (X_amount / X_volume).values.reshape([-1, 32, 1])
                     ), axis=3).reshape(-1, 32, 2)
        
        y = store['y']['y'].values
        
        store.close()
        
        split_index = int(y.shape[0] * .8)
        
        data_x_train = X[:split_index]
        data_x_val = X[split_index:]
        data_y_train = y[:split_index]
        data_y_val = y[split_index:]
        
        dataset_train = TimeSeriesDataset(data_x_train, data_y_train)
        dataset_val = TimeSeriesDataset(data_x_val, data_y_val)
        
        train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        
        model.train()
        total_size = data_y_train.shape[0]
        running_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(train_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            optimizer.zero_grad()
            
#             y_pred, h_n = model(X_batch)
            y_pred, h_n, prob, context = model(X_batch)
#             print(y_pred)
#             print(y_batch)
            loss = loss_fn(y_pred.reshape(-1), y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.shape[0]
            
            step += X_batch.shape[0]
            
#             if step % 8000 == 0:
            if idx == 1 and j % 4 == 0:
                writer.add_scalar('loss/loss', loss.item(), step)
                writer.add_scalar('loss/total_loss', running_loss / (total_size), step)

                writer.add_histogram('train/01_x_price', torchvision.utils.make_grid(X_batch[:,:,0]).data.cpu(), step)
                writer.add_histogram('train/01_x_volume', torchvision.utils.make_grid(X_batch[:,:,1]).data.cpu(), step)
#                 writer.add_histogram('train/01_x_amount', torchvision.utils.make_grid(X_batch[:,:,2]).data.cpu(), step)
                
#                 writer.add_histogram('train/02_fc1', torchvision.utils.make_grid(fc1).data.cpu(), step)
#                 writer.add_histogram('train/03_lstm1', torchvision.utils.make_grid(lstm1).data.cpu(), step)
                writer.add_histogram('train/02_h_n', torchvision.utils.make_grid(h_n).data.cpu(), step)
#                 writer.add_histogram('train/05_dropout', torchvision.utils.make_grid(dropout).data.cpu(), step)
                
#                 writer.add_histogram('train/06_encoded', torchvision.utils.make_grid(encoded).data.cpu(), step)
#                 writer.add_histogram('train/07_decoded', torchvision.utils.make_grid(decoded).data.cpu(), step)
                
                writer.add_histogram('train/07_prob', torchvision.utils.make_grid(prob).data.cpu(), step)
                writer.add_histogram('train/08_context', torchvision.utils.make_grid(context).data.cpu(), step)

    
                writer.add_histogram('train/09_pred', torchvision.utils.make_grid(y_pred).data.cpu(), step)

#         print('Epoch: [{}/{}], [{}/{}], Loss: {:.8f}'.format(epoch + 1, n_epochs, start+10, total_size, running_loss / (X_batch.shape[0] * i)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, ix, running_loss / (total_size)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size)))
        
        val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
        
        model.eval()
        eval_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(val_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            y_pred, h_n, prob, context = model(X_batch)
            loss = loss_fn(y_pred.reshape(-1), y_batch).cpu().detach().numpy()
            
            eval_loss += loss.item() * y_batch.shape[0]
        eval_loss = eval_loss / data_y_val.shape[0]
        
#         print('eval loss {:.8f}'.format(eval_loss))
        lr = scheduler.get_last_lr()[0]
        print('Epoch: [{}/{}], {}, Loss: {:.8f}, Eval loss {:.8f}, lr:{:.16f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size), eval_loss, lr))
        if eval_loss < best_mse:
            best_mse = eval_loss
            best_weights = copy.deepcopy(model.state_dict())
    
    scheduler.step()
    
# restore model and return best accuracy
model.load_state_dict(best_weights)

Epoch: [1/40], SHFE.rb.2021-04-06.5S.10T.41253.h5, Loss: 0.00333332, Eval loss 0.00048871, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-07.5S.10T.41371.h5, Loss: 0.00098996, Eval loss 0.00022761, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-08.5S.10T.41368.h5, Loss: 0.00032145, Eval loss 0.00014079, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-09.5S.10T.41366.h5, Loss: 0.00030743, Eval loss 0.00040166, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-12.5S.10T.41387.h5, Loss: 0.00049205, Eval loss 0.00012859, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-13.5S.10T.41387.h5, Loss: 0.00024008, Eval loss 0.00022601, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-14.5S.10T.41347.h5, Loss: 0.00017450, Eval loss 0.00012553, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-15.5S.10T.41386.h5, Loss: 0.00023869, Eval loss 0.00113093, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-16.5S.10T.41383.h5, Loss: 0.00023607, Eval loss 0.00012965, lr:0.

Epoch: [1/40], SHFE.rb.2021-07-22.5S.10T.41337.h5, Loss: 0.00036101, Eval loss 0.00018398, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-23.5S.10T.41252.h5, Loss: 0.00028567, Eval loss 0.00019965, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-26.5S.10T.41142.h5, Loss: 0.00034372, Eval loss 0.00017986, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-27.5S.10T.41230.h5, Loss: 0.00034127, Eval loss 0.00021121, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-28.5S.10T.41233.h5, Loss: 0.00032437, Eval loss 0.00016832, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-29.5S.10T.41270.h5, Loss: 0.00026099, Eval loss 0.00022714, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-30.5S.10T.41320.h5, Loss: 0.00049856, Eval loss 0.00090749, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-02.5S.10T.41379.h5, Loss: 0.00054936, Eval loss 0.00023675, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-03.5S.10T.41369.h5, Loss: 0.00038440, Eval loss 0.00024891, lr:0.

Epoch: [1/40], SHFE.rb.2021-11-11.5S.10T.41398.h5, Loss: 0.00058611, Eval loss 0.00031644, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-12.5S.10T.41402.h5, Loss: 0.00052538, Eval loss 0.00042573, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-15.5S.10T.41402.h5, Loss: 0.00065751, Eval loss 0.00028159, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-16.5S.10T.41402.h5, Loss: 0.00040378, Eval loss 0.00025890, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-17.5S.10T.41401.h5, Loss: 0.00035335, Eval loss 0.00020349, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-18.5S.10T.41398.h5, Loss: 0.00032485, Eval loss 0.00026963, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-19.5S.10T.41402.h5, Loss: 0.00045951, Eval loss 0.00043585, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-22.5S.10T.41390.h5, Loss: 0.00040555, Eval loss 0.00017350, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-23.5S.10T.41398.h5, Loss: 0.00039672, Eval loss 0.00026032, lr:0.

Epoch: [1/40], SHFE.rb.2022-03-02.5S.10T.41263.h5, Loss: 0.00027555, Eval loss 0.00019243, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-03.5S.10T.41333.h5, Loss: 0.00034123, Eval loss 0.00028308, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-04.5S.10T.41335.h5, Loss: 0.00028916, Eval loss 0.00016278, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-07.5S.10T.41350.h5, Loss: 0.00060427, Eval loss 0.00058551, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-08.5S.10T.41312.h5, Loss: 0.00054347, Eval loss 0.00026888, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-09.5S.10T.41202.h5, Loss: 0.00042531, Eval loss 0.00021260, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-10.5S.10T.41278.h5, Loss: 0.00030799, Eval loss 0.00017575, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-11.5S.10T.41243.h5, Loss: 0.00027113, Eval loss 0.00016870, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-14.5S.10T.41270.h5, Loss: 0.00035901, Eval loss 0.00016465, lr:0.

Epoch: [1/40], SHFE.rb.2022-06-21.5S.10T.41386.h5, Loss: 0.00026768, Eval loss 0.00016507, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-22.5S.10T.41387.h5, Loss: 0.00031986, Eval loss 0.00016847, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-23.5S.10T.41397.h5, Loss: 0.00028015, Eval loss 0.00018667, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-24.5S.10T.41391.h5, Loss: 0.00029423, Eval loss 0.00039546, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-27.5S.10T.41351.h5, Loss: 0.00019755, Eval loss 0.00014940, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-28.5S.10T.41360.h5, Loss: 0.00020146, Eval loss 0.00014149, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-29.5S.10T.41356.h5, Loss: 0.00025016, Eval loss 0.00013721, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-30.5S.10T.41384.h5, Loss: 0.00027823, Eval loss 0.00016362, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-07-01.5S.10T.41393.h5, Loss: 0.00026140, Eval loss 0.00026786, lr:0.

...

Epoch: [3/40], SHFE.rb.2021-04-28.5S.10T.41365.h5, Loss: 0.00019168, Eval loss 0.00015758, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-04-29.5S.10T.41379.h5, Loss: 0.00022615, Eval loss 0.00035998, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-04-30.5S.10T.26963.h5, Loss: 0.00021040, Eval loss 0.00012711, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-06.5S.10T.41368.h5, Loss: 0.00033174, Eval loss 0.00009831, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-07.5S.10T.41380.h5, Loss: 0.00024379, Eval loss 0.00016881, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-10.5S.10T.36712.h5, Loss: 0.00100546, Eval loss 0.00107832, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-11.5S.10T.41394.h5, Loss: 0.00081859, Eval loss 0.00040327, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-12.5S.10T.41360.h5, Loss: 0.00068633, Eval loss 0.00033366, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-05-13.5S.10T.41395.h5, Loss: 0.00211075, Eval loss 0.00042076, lr:0.

Epoch: [3/40], SHFE.rb.2021-08-13.5S.10T.41364.h5, Loss: 0.00029379, Eval loss 0.00019813, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-16.5S.10T.41370.h5, Loss: 0.00041390, Eval loss 0.00021297, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-17.5S.10T.41360.h5, Loss: 0.00023293, Eval loss 0.00021740, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-18.5S.10T.41402.h5, Loss: 0.00045923, Eval loss 0.00086877, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-19.5S.10T.41394.h5, Loss: 0.00055299, Eval loss 0.00028027, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-20.5S.10T.41397.h5, Loss: 0.00037215, Eval loss 0.00029186, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-23.5S.10T.41398.h5, Loss: 0.00041414, Eval loss 0.00026696, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-24.5S.10T.41383.h5, Loss: 0.00028444, Eval loss 0.00019073, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-08-25.5S.10T.41393.h5, Loss: 0.00030075, Eval loss 0.00020653, lr:0.

Epoch: [3/40], SHFE.rb.2021-12-03.5S.10T.41395.h5, Loss: 0.00032702, Eval loss 0.00017380, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-06.5S.10T.41383.h5, Loss: 0.00025818, Eval loss 0.00015275, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-07.5S.10T.41380.h5, Loss: 0.00024237, Eval loss 0.00014771, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-08.5S.10T.41400.h5, Loss: 0.00031797, Eval loss 0.00017361, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-09.5S.10T.41388.h5, Loss: 0.00030012, Eval loss 0.00016763, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-10.5S.10T.41376.h5, Loss: 0.00024508, Eval loss 0.00013840, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-13.5S.10T.41372.h5, Loss: 0.00027516, Eval loss 0.00012440, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-14.5S.10T.41352.h5, Loss: 0.00017458, Eval loss 0.00013083, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2021-12-15.5S.10T.41346.h5, Loss: 0.00019568, Eval loss 0.00013297, lr:0.

Epoch: [3/40], SHFE.rb.2022-03-24.5S.10T.40924.h5, Loss: 0.00017679, Eval loss 0.00016128, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-03-25.5S.10T.40722.h5, Loss: 0.00014588, Eval loss 0.00013403, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-03-28.5S.10T.40074.h5, Loss: 0.00023261, Eval loss 0.00010820, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-03-29.5S.10T.41190.h5, Loss: 0.00021739, Eval loss 0.00013151, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-03-30.5S.10T.41183.h5, Loss: 0.00016019, Eval loss 0.00013280, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-03-31.5S.10T.41284.h5, Loss: 0.00019472, Eval loss 0.00012588, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-04-01.5S.10T.26937.h5, Loss: 0.00018694, Eval loss 0.00012342, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-04-06.5S.10T.41192.h5, Loss: 0.00029774, Eval loss 0.00017969, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-04-07.5S.10T.41322.h5, Loss: 0.00020004, Eval loss 0.00014517, lr:0.

KeyboardInterrupt: 

In [44]:
model.load_state_dict(best_weights)

<All keys matched successfully>

In [45]:
y_pred

tensor([[-0.0902],
        [ 0.0082],
        [-0.0651],
        [-0.1312],
        [ 0.0557],
        [-0.0268],
        [-0.0085],
        [-0.1070]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [52]:
# torch.save(model.state_dict(), 'D:/option/models/model_0705_autoencoder_gru_dims.pt')
torch.save(model.state_dict(), 'D:/option/models/model_0705_autoencoder_attn_dims.pt')

In [46]:
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_lstm4.pt'))

In [46]:
device = 0

In [47]:
# TEST_DIR = os.path.join('D:/option/l8_autoencoder', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/dims/diff', '%s_test' % DATA_PATH)

In [49]:
def plot_file(p):
    store = pd.HDFStore(p, 'r')
    
    X_price = store['X_price']
    X_volume = store['X_volume']
    X_amount = store['X_amount']
        
    X_test = np.stack((X_price.values.reshape([-1, 32, 1]), 
                  X_volume.values.reshape([-1, 32, 1]), 
#                   X_amount.values.reshape([-1, 32, 1]), 
#                   (X_amount / X_volume).values.reshape([-1, 32, 1])
                 ), axis=3).reshape(-1, 32, 2)
        
    y_test = store['y']['y'].values
    
    price = store['price']
    
    store.close()
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred, h_n, prob, context = model(X_batch.to(0))
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)

    fig, ax1 = plt.subplots()
    
    ax1.plot(y_pred, label='pred', color='b', marker='>')
    ax1.plot(y_test, label='real', color='r', marker='<')

    ax2 = ax1.twinx()
    ax2.plot(price['price'].values, color='g', label='price', marker='.')

    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.show()

In [50]:
%matplotlib qt

In [53]:
plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.5S.10T.41388.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-06-07.5S.10T.41405.h5'))



In [30]:
%matplotlib inline

In [31]:
# TEST_DIR = os.path.join('D:/option/', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/diff', '%s_test' % DATA_PATH)

In [33]:
for f in os.listdir(TEST_DIR):
    print(f)
    store = pd.HDFStore(os.path.join(TEST_DIR, f), 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
#     y_inverse = df_y['y'] * df_y['std'] + df_y['mu']
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred = model(X_batch.to(0)) # , fc1, lstm1, h_n, dropout
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
    
#     plt.plot(y_pred, label='pred')
#     plt.plot(y_test, label='real')
#     plt.legend()
#     plt.savefig(os.path.join('D:/option/', '%s_test_fig_lstm4' % DATA_PATH, '%s.jpg' % f), dpi=160)
#     plt.close()
    fig, ax1 = plt.subplots()

    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
#     ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.savefig(os.path.join('D:/option/master/diff', '%s_test_fig' % DATA_PATH, '%s.jpg' % f), dpi=160)
    plt.close()
    

SHFE.rb.2023-05-04.15S.15T.20704.h5
SHFE.rb.2023-05-05.15S.15T.20704.h5
SHFE.rb.2023-05-08.15S.15T.20704.h5
SHFE.rb.2023-05-09.15S.15T.20704.h5
SHFE.rb.2023-05-10.15S.15T.20704.h5
SHFE.rb.2023-05-11.15S.15T.20704.h5
SHFE.rb.2023-05-12.15S.15T.20704.h5
SHFE.rb.2023-05-15.15S.15T.20704.h5
SHFE.rb.2023-05-16.15S.15T.20704.h5
SHFE.rb.2023-05-17.15S.15T.20704.h5
SHFE.rb.2023-05-18.15S.15T.20704.h5
SHFE.rb.2023-05-19.15S.15T.20704.h5
SHFE.rb.2023-05-22.15S.15T.20704.h5
SHFE.rb.2023-05-23.15S.15T.20704.h5
SHFE.rb.2023-05-24.15S.15T.20704.h5
SHFE.rb.2023-05-25.15S.15T.20704.h5
SHFE.rb.2023-05-26.15S.15T.20704.h5
SHFE.rb.2023-05-29.15S.15T.20704.h5
SHFE.rb.2023-05-30.15S.15T.20704.h5
SHFE.rb.2023-05-31.15S.15T.20704.h5
SHFE.rb.2023-06-01.15S.15T.20704.h5
SHFE.rb.2023-06-02.15S.15T.20704.h5
SHFE.rb.2023-06-05.15S.15T.20704.h5
SHFE.rb.2023-06-06.15S.15T.20704.h5
SHFE.rb.2023-06-07.15S.15T.20704.h5
SHFE.rb.2023-06-08.15S.15T.20704.h5
SHFE.rb.2023-06-09.15S.15T.20704.h5


### l8_30S_15T_mean 0.00028